
# Audio Data Augmentation Script

This Python script performs audio data augmentation using the **Librosa** library to enhance a dataset of `.wav` files. The script includes functionalities to apply various augmentations such as adding noise, changing pitch, time-stretching, altering speed, and flipping audio. The augmented audio files are saved in a specified output directory.

## Features
1. **Load Audio Files**: Load `.wav` files with their original sample rate.
2. **Save Augmented Audio**: Save the augmented audio files in the `int16` format.
3. **Augmentation Techniques**:
   - **Add Noise**: Adds Gaussian noise at different levels.
   - **Time Stretching**: Changes the speed of audio using resampling.
   - **Pitch Shifting**: Shifts pitch by a specified number of steps.
   - **Speed Change**: Alters playback speed by a factor.
   - **Audio Flipping**: Reverses the audio waveform.

   
 By Dheer N Raijada

In [2]:
!pip install resampy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.5 MB/s eta 0:00:0000:0100:01


In [3]:
import os
import librosa
import numpy as np
import resampy
from scipy.io.wavfile import write

# Function to load audio
def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Load with original sample rate
    return y, sr

# Function to save augmented audio
def save_audio(file_name, y, sr):
    scaled = np.int16(y / np.max(np.abs(y)) * 32767)  # Scale audio to int16
    write(file_name, sr, scaled)

# Augmentation functions
def add_noise(y, noise_level=0.02):
    return y + np.random.normal(0, noise_level, y.shape)

def custom_time_stretch(y, rate):
    # Handle time stretching using resampling
    stretched = librosa.resample(y, orig_sr=1.0, target_sr=rate, res_type="kaiser_best")
    return stretched



def pitch_shift(y, sr, n_steps):
    return librosa.effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)

def change_speed(y, speed_factor):
    return librosa.effects.time_stretch(y, rate=speed_factor)


def flip_audio(y):
    return np.flip(y)

# Directory paths
input_dir = "/kaggle/input/dheer-audio"  # Replace with your folder containing .wav files
output_dir = "./output/augmented_audio"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Augmentation parameters
noise_levels = [0.01, 0.02, 0.03, 0.04, 0.05]
time_stretch_rates = [0.5, 1.2, 1.4, 1.8, 2.0]
pitch_shift_steps = [-4, -2, 0, 2, 4]
speed_factors = [0.5, 1.2, 1.5, 1.8, 2.0]

# Process files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".wav"):
        file_path = os.path.join(input_dir, file_name)
        y, sr = load_audio(file_path)
        
        # Generate augmented audio samples
        for i, noise_level in enumerate(noise_levels):
            noise_aug = add_noise(y, noise_level)
            save_audio(f"{output_dir}/{file_name}_noise_{i+1}.wav", noise_aug, sr)
        
        for i, rate in enumerate(time_stretch_rates):
            try:
                time_aug = custom_time_stretch(y, rate)
                save_audio(f"{output_dir}/{file_name}_time_stretch_{i+1}.wav", time_aug, sr)
            except Exception as e:
                print(f"Skipping time stretch with rate {rate} for {file_name}: {e}")
        
        for i, n_steps in enumerate(pitch_shift_steps):
            try:
                pitch_aug = pitch_shift(y, sr, n_steps)
                save_audio(f"{output_dir}/{file_name}_pitch_shift_{i+1}.wav", pitch_aug, sr)
            except Exception as e:
                print(f"Skipping pitch shift with steps {n_steps} for {file_name}: {e}")
        
        for i, speed_factor in enumerate(speed_factors):
            try:
                speed_aug = change_speed(y, speed_factor)
                save_audio(f"{output_dir}/{file_name}_speed_change_{i+1}.wav", speed_aug, sr)
            except Exception as e:
                print(f"Skipping speed change with factor {speed_factor} for {file_name}: {e}")
        
        for i in range(5):  # Flip doesn't need parameters, just generate 5 copies
            flip_aug = flip_audio(y)
            save_audio(f"{output_dir}/{file_name}_flip_{i+1}.wav", flip_aug, sr)

        print(f"Augmented files for {file_name} saved in {output_dir}")


Augmented files for DN1F30.wav saved in ./output/augmented_audio
